In [ ]:
from turtle import forward
import numpy as np
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.utils.data as Data
import data_loader as dl
import train as tt
import os
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
datapath="data"

In [ ]:
net=torch.load("generate_data/model/100.pt")

In [ ]:
item = os.listdir(datapath)
data=dl.data_loader(dic=datapath,file=item[0],filetype=2,state="normal")# use prepared if already add scarfford by runing Addscarford.ipynb 
print(item)

if you want less MSE, you can we can use several rna (10 for example) to fine tune the model. only adjust a b in the step, won't change the spearman correlation
or use the model dierctly(commnet the following codes).

In [ ]:
finetune,data = Data.random_split(data,[10,len(data)-10]) # choose 10 seq to finetune
valoader=Data.DataLoader(dataset=finetune ,batch_size=1000,shuffle=True)
paras =[]
paraname =[]
lossf=nn.SmoothL1Loss(reduction="mean")
for name,para in net.named_parameters():
    if name.startswith('embed3'):
        paras.append(para)
        paraname.append(name)
opt=torch.optim.Adam(paras,lr=0.0001)
she=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt,T_0=5)   

step=1000 #finetune step

net.train()
for i in range(step):
    for seq,typ,eff in valoader:
        seq=seq.cuda()
        typ=typ.cuda()
        eff=eff.cuda()
        eff+=(torch.rand(size=eff.size(),device="cuda")-0.5)*0.1
        eff =torch.abs(eff)
        pre=net(seq,typ)
        loss=lossf(pre,eff)
        opt.zero_grad()
        loss.backward()
        opt.step()
        she.step()

In [ ]:
valoader=Data.DataLoader(dataset=data ,batch_size=1,shuffle=True)
for sc in item:
    net.eval()
    for i in item:
        presave=[]
        effsave=[]
        for seq,typ,eff in valoader:
            seq=seq.cuda()
            typ=typ.cuda()
            eff=eff.cuda()
            pre=net(seq,typ,train=False)
            pre=pre.detach().cpu().numpy()
            eff=eff.detach().cpu().numpy()
            if len(presave)==0:
                presave=np.copy(pre)
                effsave=np.copy(eff)
            else:
                presave=np.concatenate((presave,pre))
                effsave=np.concatenate((effsave,eff))
        plt.scatter(presave,effsave,s=1)
        print(ss.spearmanr(effsave.flatten(),presave.flatten())[0])
        print(len(presave))
        plt.show()
